# 数据准备

## 数据导入

In [27]:
import pandas as pd
def load_data(path, add_time=True, describe=True):
    data = pd.read_csv(path)
    if describe:
        display(data.describe())
    if add_time:
        data['Date'] = data['Date'].apply(lambda data: data.split('/'))
        data['Year'] = data['Date'].apply(lambda data: int(data[2].split(' ')[0]))
        data['Month'] = data['Date'].apply(lambda data: int(data[1]))
        data['Day'] = data['Date'].apply(lambda data: int(data[0]))
        data['Hour'] = data['Date'].apply(lambda data: int(data[2].split(' ')[1].split(':')[0]))
        data.drop('Date', inplace=True, axis=1)
        data = data[['Year', 'Month', 'Day','Hour','wind_speed', 'wind_direction', 'wind_power']]
    return data

In [39]:
path = '/Users/apple/Documents/ML_Project/ML - 2.1/Data/国际西班牙数据.csv'
data = load_data(path, add_time=True, describe=False)

输出连续区间

In [3]:
index = data[data['wind_speed'].isna() |
    data['wind_direction'].isna() |
    data['wind_power'].isna()]['wind_power'].index.tolist()
a = index[0]
b=-1
for i,x in enumerate(index):
    if i<len(index)-1:
        if index[i+1] > index[i]+1:
            print('Continue: [',b+1, ',', a-1,']','len:',a-b-2,
                '\tNan: [',a, ',',index[i], '] len:',index[i]-a+1)     
            a = index[i+1]
            b = index[i]
    else: 
        a=index[-15]
        print('Continue: [',b+1, ',', a-1,']','len:',a-b-2,
                '\tNan: [',a, ',',index[i], '] len:',index[i]-a+1)     
        a=len(data)
        b=index[i]
        print('Continue: [',b+1, ',', a-1,']','len:',a-b-2)


Continue: [ 0 , 975 ] len: 975 	Nan: [ 976 , 976 ] len: 1
Continue: [ 977 , 2216 ] len: 1239 	Nan: [ 2217 , 2221 ] len: 5
Continue: [ 2222 , 3498 ] len: 1276 	Nan: [ 3499 , 3560 ] len: 62
Continue: [ 3561 , 3572 ] len: 11 	Nan: [ 3573 , 3589 ] len: 17
Continue: [ 3590 , 4314 ] len: 724 	Nan: [ 4315 , 4401 ] len: 87
Continue: [ 4402 , 6255 ] len: 1853 	Nan: [ 6256 , 6273 ] len: 18
Continue: [ 6274 , 6375 ] len: 101 	Nan: [ 6376 , 6376 ] len: 1
Continue: [ 6377 , 6417 ] len: 40 	Nan: [ 6418 , 6425 ] len: 8
Continue: [ 6426 , 10427 ] len: 4001 	Nan: [ 10428 , 10447 ] len: 20
Continue: [ 10448 , 13432 ] len: 2984 	Nan: [ 13433 , 13434 ] len: 2
Continue: [ 13435 , 13976 ] len: 541 	Nan: [ 13977 , 13985 ] len: 9
Continue: [ 13986 , 14000 ] len: 14 	Nan: [ 14001 , 14009 ] len: 9
Continue: [ 14010 , 14024 ] len: 14 	Nan: [ 14025 , 14033 ] len: 9
Continue: [ 14034 , 14048 ] len: 14 	Nan: [ 14049 , 14057 ] len: 9
Continue: [ 14058 , 14072 ] len: 14 	Nan: [ 14073 , 14077 ] len: 5
Continue: [ 1407

## 可视化

In [61]:
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
def plot_module1(year, month, day, data, figsize=(14,16), save_fig=False, close_fig=True):
    data['ws_sin(wd)'] = data['wind_speed']*np.sin(data['wind_direction'])
    data['ws_cos(wd)'] = data['wind_speed']*np.cos(data['wind_direction'])
    data['ws_wd'] = data['wind_speed']*data['wind_direction']
    data['ws(Scaled)_wd(Scaled)'] = \
    MinMaxScaler().fit_transform(data['wind_speed'].values.reshape(-1,1))*\
    MinMaxScaler().fit_transform(data['wind_direction'].values.reshape(-1,1))

    data['wind_power(Scaled)'] = \
    MinMaxScaler().fit_transform(data['wind_power'].values.reshape(-1,1))
    df = data
    fig, axes = plt.subplots(4,2,figsize=figsize)
    fig.suptitle('Year\Month\Day: {}\{}\{}'.format(year,month,day), fontsize=15)
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_speed'].plot(ax=axes[0,0],title='wind_speed', ylim=[0,7])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_direction'].plot(ax=axes[1,0],title='wind_direction', ylim=[90,360])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_power'].plot(ax=axes[2,0], title='wind_power', ylim=[0,5000])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['ws_sin(wd)'].plot(ax=axes[0,1],title='wind_speed*sin(wind_direction)')
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['ws_cos(wd)'].plot(ax=axes[1,1],title='wind_speed * cos(wind_direction)')
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['ws_wd'].plot(ax=axes[2,1],title='wind_speed*wind_direction')
    # df[(df['Month']==month) & (df['Day']==day)]['ws(Scaled)_wd(Scaled)'].plot(ax=axes[3,1],title='wind_speed*wind_direction(Both Scaled)')


    pd.Series(MinMaxScaler().fit_transform\
    (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
     ['wind_speed'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[3,0])

    pd.Series(MinMaxScaler().fit_transform\
    (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
     ['wind_direction'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[3,0],title='wind_speed & wind_direction')
    if save_fig:
        plt.savefig('/Users/apple/Documents/ML_Project/ML - 2.1/figure/{}\{}\{}.png'.format(year,month,day))
    if close_fig:
        plt.close()
def plot_module2(year, month, day, data, figsize=(14,14), save_fig=False, close_fig=True):
    data['ws_sin(wd)'] = data['wind_speed']*np.sin(data['wind_direction'])
    data['ws_cos(wd)'] = data['wind_speed']*np.cos(data['wind_direction'])
    data['ws_wd'] = data['wind_speed']*data['wind_direction']
    data['ws(Scaled)_wd(Scaled)'] = \
    MinMaxScaler().fit_transform(data['wind_speed'].values.reshape(-1,1))*\
    MinMaxScaler().fit_transform(data['wind_direction'].values.reshape(-1,1))

    data['wind_power(Scaled)'] = \
    MinMaxScaler().fit_transform(data['wind_power'].values.reshape(-1,1))
    df = data
    fig, axes = plt.subplots(3,2,figsize=figsize)
    fig.suptitle('Year\Month\Day: {}\{}\{}'.format(year,month,day), fontsize=15)
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_speed'].plot(ax=axes[0,0],title='wind_speed', ylim=[0,7])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_direction'].plot(ax=axes[1,0],title='wind_direction', ylim=[90,360])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_power'].plot(ax=axes[2,0], title='wind_power', ylim=[0,5000])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_power(Scaled)'].plot(ax=axes[1,1],title='wind_power(Scaled)')
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['ws_wd'].plot(ax=axes[2,1],title='wind_speed * wind_direction')
    
    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
               ['wind_speed'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1])

    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
               ['wind_direction'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1],title='wind_speed(Scaled) & wind_direction(Scaled)')
    if save_fig:
        plt.savefig('/Users/apple/Documents/ML_Project/ML - 2.1/figure/{}\{}\{}.png'.format(year,month,day))
    if close_fig:
        plt.close()

In [69]:
from tqdm.notebook import tqdm as tqdm
for day in tqdm(np.arange(1,2)):
    plot_module1(year=2017, month=10, day=day, data=data, 
                 figsize=(8,13), save_fig=False, close_fig=True)
for day in tqdm(np.arange(1,2)):
    plot_module2(year=2017, month=10, day=day, data=data, 
                 figsize=(8,10), save_fig=False, close_fig=True)

## 数据扩增

In [21]:
def Data_Extend_fun(Data, hour_num, columns):
    if hour_num==0:
        return Data
    elif hour_num>0:
        Extend_data = Data[hour_num:len(Data)].copy()
        Extend_data.index = np.arange(0,len(Extend_data)).tolist()
        for index in columns:
            for i in np.arange(1,hour_num+1):
                a = Data[index].iloc[(hour_num-i):-i].copy()
                a.index = np.arange(0,len(Extend_data)).tolist()
                Extend_data[index+'-{}'.format(i)] = a
        return Extend_data
    else:
        print('ERROR: hour num cannot be negative!')

In [38]:
from sklearn.preprocessing import MinMaxScaler
def get_data(Data, 
             hour_num=0, 
             columns=['wind_speed', 'wind_direction', 'wind_power'], 
             train_index=[6426,10427],
             test_index=[14389,17872],
             drop_time=True,
             scale=True):
    Train = Data_Extend_fun(Data=data.iloc[train_index[0]:train_index[1]],
                            hour_num=hour_num,columns = columns)
    Test = Data_Extend_fun(Data=data.iloc[test_index[0]:test_index[1]],
                           hour_num=hour_num,columns = columns)

    X_train = Train.drop('wind_power', axis=1)
    X_test = Test.drop('wind_power', axis=1)
    Y_train = Train['wind_power']
    Y_test = Test['wind_power']

    if drop_time:
        X_train.drop(['Year', 'Month', 'Day','Hour'], axis=1)
        X_test.drop(['Year', 'Month', 'Day','Hour'], axis=1)
        
    if scale:
        X_Scaler = MinMaxScaler()
        X_train = X_Scaler.fit_transform(X_train)
        X_test = X_Scaler.transform(X_test)

        Y_Scaler = MinMaxScaler()
        Y_train = Y_Scaler.fit_transform(Y_train.values.reshape(-1,1)).reshape(len(Y_train),)
        Y_test = Y_Scaler.transform(Y_test.values.reshape(-1,1)).reshape(len(Y_test),)
    return X_train, X_test, Y_train, Y_test

In [43]:
X_train, X_test, Y_train, Y_test = get_data(data, hour_num=3, 
                                            drop_time=True,scale=True)

不扩增优于扩增

# 模型预测

In [28]:
from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_error
def model_test(Base, X_train, X_test, Y_train, Y_test, 
               n_estimators=500, verbose_eval=100):
    ngb = NGBRegressor(Base=Base, 
                       n_estimators=n_estimators,
                       verbose_eval=verbose_eval)
    print(ngb,'\n')
    ngb.fit(X_train, Y_train)

    Y_preds = ngb.predict(X_test)
    Y_dists = ngb.pred_dist(X_test) # return norm method: mean std

    # test Mean Squared Error
    test_MSE = mean_squared_error(Y_preds, Y_test)
    print('\nTest MSE', test_MSE)

    # test Negative Log Likelihood
    test_NLL = -Y_dists.logpdf(Y_test).mean()
    print('Test NLL', test_NLL)

1. linear_svr_learner（0.0245）
2. esn_linear_svr_learner（0.0275）
3. default_linear_learner（0.0312）
4. default_tree_learner（0.0425）
5. esn_kernel_ridge_learner（0.0442）
6. kernel_ridge_learner（0.0478）
7. esn_ridge_learner（0.0482）

## default_linear_learner

In [41]:
from ngboost.learners import default_linear_learner
model_test(Base=default_linear_learner(alpha=0.1),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=100)

NGBRegressor(Base=Ridge(alpha=0.1, copy_X=True, fit_intercept=True,
                        max_iter=None, normalize=False, random_state=None,
                        solver='auto', tol=0.001),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=100) 

[iter 0] loss=0.0536 val_loss=0.0000 scale=0.5000 norm=0.2703
[iter 100] loss=-0.3436 val_loss=0.0000 scale=0.1250 norm=0.0571
[iter 200] loss=-0.3690 val_loss=0.0000 scale=0.0312 norm=0.0139
[iter 300] loss=-0.3706 val_loss=0.0000 scale=0.0078 norm=0.0034
[iter 400] loss=-0.3707 val_loss=0.0000 scale=0.0020 norm=0.0009

Test MSE 0.031239293219421656
Test NLL -0.3310202514535008


## default_tree_learner

In [32]:
from ngboost.learners import default_tree_learner
model_test(Base=default_tree_learner(depth=6),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=1000, verbose_eval=200)

NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=6,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=None,
                                        splitter='best'),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=1000, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=200) 

[iter 0] loss=0.0537 val_loss=0.0000 scale=0.5000 norm=0.2702
[iter 200] loss=-0.3028 val_loss=0.0000 scale=0.1250 norm=0.0480
[iter 400] loss=-0.3134 val_loss=0.000

## linear_svr_learner

In [31]:
from ngboost.learners import linear_svr_learner
model_test(Base=linear_svr_learner(epsilon=0.0, 
                                   C=0.05, 
                                   max_iter=10000),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=100)

NGBRegressor(Base=LinearSVR(C=0.05, dual=True, epsilon=0.0, fit_intercept=True,
                            intercept_scaling=1.0, loss='epsilon_insensitive',
                            max_iter=1000, random_state=None, tol=0.0001,
                            verbose=0),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=100) 

[iter 0] loss=0.0537 val_loss=0.0000 scale=1.0000 norm=0.5405
[iter 100] loss=-0.2473 val_loss=0.0000 scale=0.2500 norm=0.1247
[iter 200] loss=-0.2932 val_loss=0.0000 scale=0.0625 norm=0.0300
[iter 300] loss=-0.2994 val_loss=0.0000 scale=0.0156 norm=0.0072
[iter 400] loss=-0.2999 val_loss=0.0000 scale=0.0078 norm=0.0036

Test MSE 0.024508858474129803
Test NLL -0.3773893376173749


## kernel_ridge_learner

In [11]:
from ngboost.learners import kernel_ridge_learner
model_test(Base=kernel_ridge_learner(alpha=0.5, 
                                    kernel="poly",
                                    degree=3),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=10)

NGBRegressor(Base=KernelRidge(alpha=0.5, coef0=1, degree=3, gamma=None,
                              kernel='poly', kernel_params=None),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=10) 

[iter 0] loss=0.0537 val_loss=0.0000 scale=1.0000 norm=0.5405
[iter 10] loss=-0.0879 val_loss=0.0000 scale=0.5000 norm=0.2335
[iter 20] loss=-0.1340 val_loss=0.0000 scale=0.5000 norm=0.2299
[iter 30] loss=-0.1735 val_loss=0.0000 scale=0.5000 norm=0.2303
[iter 40] loss=-0.2086 val_loss=0.0000 scale=0.5000 norm=0.2329
[iter 50] loss=-0.2354 val_loss=0.0000 scale=0.2500 norm=0.1180
[iter 60] loss=-0.2499 val_loss=0.0000 scale=0.2500 norm=0.1189
[iter 70] loss=-0.2633 val_loss=0.0000 scale=0.2500 norm=0.1198
[iter 80] loss=-0.2756 val_loss=0.0000 scale=0.2500 norm=0.1206
[iter 90] loss=-0.2865 val_lo

## esn_ridge_learner

In [152]:
from ngboost.learners import esn_ridge_learner
model_test(Base=esn_ridge_learner(n_readout=1000,
                                  n_components=100,
                                  alpha=0.01),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=10)

NGBRegressor(Base=<function esn_ridge_learner at 0x1a191217b8>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=10) 

[iter 0] loss=0.0535 val_loss=0.0000 scale=1.0000 norm=0.5407
[iter 10] loss=-0.0623 val_loss=0.0000 scale=1.0000 norm=0.4658
[iter 20] loss=-0.1406 val_loss=0.0000 scale=1.0000 norm=0.4331
[iter 30] loss=-0.2038 val_loss=0.0000 scale=1.0000 norm=0.4176
[iter 40] loss=-0.2585 val_loss=0.0000 scale=1.0000 norm=0.4105
[iter 50] loss=-0.3082 val_loss=0.0000 scale=1.0000 norm=0.4077
[iter 60] loss=-0.3543 val_loss=0.0000 scale=1.0000 norm=0.4072
[iter 70] loss=-0.3978 val_loss=0.0000 scale=1.0000 norm=0.4078
[iter 80] loss=-0.4392 val_loss=0.0000 scale=1.0000 norm=0.4090
[iter 90] loss=-0.4788 val_loss=0.0000 scale=1.0000 norm=0.4102
[iter 100] loss=-0.5170 val_loss=0.0000

## esn_kernel_ridge_learner

In [12]:
from ngboost.learners import esn_kernel_ridge_learner
model_test(Base=esn_kernel_ridge_learner(n_readout=1000,
                                         n_components=100,
                                         alpha=1, 
                                         kernel='poly',
                                         degree=3),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=5)

NGBRegressor(Base=<ngboost.esn_learners.ESN_kernel_ridge_learner object at 0x1a1e7d39e8>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=5) 

[iter 0] loss=0.0537 val_loss=0.0000 scale=1.0000 norm=0.5405
[iter 5] loss=-0.0293 val_loss=0.0000 scale=1.0000 norm=0.4918
[iter 10] loss=-0.0767 val_loss=0.0000 scale=0.5000 norm=0.2369
[iter 15] loss=-0.1013 val_loss=0.0000 scale=0.5000 norm=0.2340
[iter 20] loss=-0.1238 val_loss=0.0000 scale=0.5000 norm=0.2325
[iter 25] loss=-0.1447 val_loss=0.0000 scale=0.5000 norm=0.2320
[iter 30] loss=-0.1642 val_loss=0.0000 scale=0.5000 norm=0.2322
[iter 35] loss=-0.1826 val_loss=0.0000 scale=0.5000 norm=0.2331
[iter 40] loss=-0.1999 val_loss=0.0000 scale=0.5000 norm=0.2344
[iter 45] loss=-0.2163 val_loss=0.0000 scale=0.5000 norm=0.2360
[iter 50] loss=

## esn_linear_svr_learner

In [15]:
from ngboost.learners import esn_linear_svr_learner
model_test(Base=esn_linear_svr_learner(n_readout=1000,
                                         n_components=100,
                                         epsilon=0.0,
                                         C=0.02,
                                         max_iter=1000),
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=10)

NGBRegressor(Base=<ngboost.esn_learners.ESN_linear_svr_learner object at 0x1a1e846e48>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=10) 

[iter 0] loss=0.0537 val_loss=0.0000 scale=0.5000 norm=0.2702
[iter 10] loss=0.0326 val_loss=0.0000 scale=0.5000 norm=0.2720
[iter 20] loss=0.0090 val_loss=0.0000 scale=0.5000 norm=0.2737
[iter 30] loss=-0.0174 val_loss=0.0000 scale=0.5000 norm=0.2754
[iter 40] loss=-0.0421 val_loss=0.0000 scale=0.5000 norm=0.2789
[iter 50] loss=-0.0671 val_loss=0.0000 scale=0.5000 norm=0.2828
[iter 60] loss=-0.0858 val_loss=0.0000 scale=0.2500 norm=0.1430
[iter 70] loss=-0.0970 val_loss=0.0000 scale=0.2500 norm=0.1442
[iter 80] loss=-0.1085 val_loss=0.0000 scale=0.2500 norm=0.1453
[iter 90] loss=-0.1192 val_loss=0.0000 scale=0.2500 norm=0.1465
[iter 100] loss=-